In [ ]:
!pip install noisereduce

In [ ]:
!pip install pydub

In [ ]:
import os
import librosa
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from scipy.signal import welch, butter, lfilter
import noisereduce as nr
from pydub import AudioSegment
from pydub.effects import normalize

In [ ]:
def reduce_noise(audio, sample_rate):
  return nr.reduce_noise(y=audio, sr=sample_rate)

In [ ]:
# Функция для извлечения признаков
def extract_features(audio, sample_rate, channel):
  if channel is None:
    # моно
    audio = audio.mean(axis=0)
  elif channel < audio.shape[0]:
    # иначе извлекаем канал
    audio = audio[channel]

  # Удаление шума
  audio = reduce_noise(audio=audio, sample_rate=sample_rate)

  # Извлечение мелспектрограммы
  mel_spectrogram = librosa.feature.melspectrogram(y=audio, sr=sample_rate, n_mels=40, fmax=8000)
  mel_spectrogram_db = librosa.power_to_db(mel_spectrogram, ref=np.max)
  mel_features = np.mean(mel_spectrogram_db.T, axis=0)

  # Извлечение спектра Уэльса (PSD)
  freqs, psd = welch(audio, fs=sample_rate)

  # Объединение признаков в один вектор
  combined_features = np.hstack((mel_features, psd))

  return combined_features

# Парсинг данных

Оригинальные данные имеют следующую файловую структуру

![Файловая структура](files.jpg "Файловая структура")

Рассмотрев статью от создателей датасета https://arxiv.org/pdf/1909.09347, мы выяснили, что звук в датасете восьмиканальный, собранный с восьми микрофонов, расположенных по кругу на расстоянии 45 градусов между друг другом. В этой же статье было приведено положение определяемых нами объектов в пространстве. Благодаря этому мы смогли найти микрофон, а соответственно и канал наиболее близкий к каждому объекту, что позволило получить лучшее качество звука и модели. 

Таким образом, клапану соответствует канал 0, насосу - 2, вентилятору - 4, направляющей - 6.

In [ ]:
root_dir = 'data'

data = []

name_mapping = {
  'valve': 0,
  'pump': 2,
  'fan': 4,
  'slider': 6
}

Парсер выделяет в данных следующие поля:

- db - значение децибел, при котором был записан звук
- chanel - канал, соответствующий типу оборудования
- id - идентификатор звука
- is_norm - является ли звук номальным или аномальным (1 или 0)
- file_name - название .wav файла
- mel_[i] - 40 признаков звука, выделенные с помощью мелспектрограмм
- psd_[i] - 129 признаков, выделенные с помощью спектра Уэльса

In [ ]:
# Прасинг данных с монозвуком

for db in os.listdir(root_dir):
  db_path = os.path.join(root_dir, db)
  if os.path.isdir(db_path):
    for name in os.listdir(db_path):
      name_path = os.path.join(db_path, name)
      if os.path.isdir(name_path):
        for id_ in os.listdir(name_path):
          id_path = os.path.join(name_path, id_)
          if os.path.isdir(id_path):
            for norm in os.listdir(id_path):
              norm_path = os.path.join(id_path, norm)
              if os.path.isdir(norm_path):
                for fn in os.listdir(norm_path):
                  if fn.endswith('.wav'):
                    is_norm_value = 0 if norm == 'abnormal' else 1
                    file_path = os.path.join(norm_path, fn)
                    features = extract_features(file_path)
                    data.append({
                      'db': db,
                      'chanel': name_mapping.get(name, name),
                      'id': id_,
                      'is_norm': is_norm_value,
                      'file_name': fn,
                      **{f'mel{i+1}': features[i] for i in range(len(features)-1)},
                      'psd': features[-1]
                    })

df = pd.DataFrame(data)

df.to_csv('data_mono.csv', index=False)

In [ ]:
# Прасинг данных с выделением для каждого типа оборудования только соответствующего кананла
 
for db in os.listdir(root_dir):
  db_path = os.path.join(root_dir, db)
  if os.path.isdir(db_path):
    for name in os.listdir(db_path):
      name_path = os.path.join(db_path, name)
      if os.path.isdir(name_path):
        for id_ in os.listdir(name_path):
          id_path = os.path.join(name_path, id_)
          if os.path.isdir(id_path):
            for norm in os.listdir(id_path):
              norm_path = os.path.join(id_path, norm)
              if os.path.isdir(norm_path):
                for fn in os.listdir(norm_path):
                  if fn.endswith('.wav'):
                    is_norm_value = 0 if norm == 'abnormal' else 1
                    file_path = os.path.join(norm_path, fn)
                    chanel = name_mapping.get(name, name)
                    features = extract_features(file_path, chanel)
                    data.append({
                      'db': db,
                      'chanel': chanel,
                      'id': id_,
                      'is_norm': is_norm_value,
                      'file_name': fn,
                      **{f'mel{i+1}': features[i] for i in range(len(features)-1)},
                      'psd': features[-1]
                    })

df = pd.DataFrame(data)

df.to_csv('data_stereo.csv', index=False)